<img width="8%" alt="Google Sheets.png" src="https://raw.githubusercontent.com/jupyter-naas/awesome-notebooks/master/.github/assets/logos/Google%20Sheets.png" style="border-radius: 15%">

# Google Sheets - Update deals database

**Tags:** #googlesheets #gsheet #data #naas_drivers #sales-engine #automation #deals

**Author:** [Florent Ravenel](https://www.linkedin.com/in/florent-ravenel/)

**Description:** This notebook updates "Deals" database.

## Input

### Import libraries

In [1]:
from naas_drivers import gsheet
import pandas as pd
import os
from datetime import date
import naas_data_product
import time

✅ utils file '/home/ftp/abi/utils/data.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/llm.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_chat_plugin.ipynb' successfully loaded.
✅ utils file '/home/ftp/abi/utils/naas_lab.ipynb' successfully loaded.


### Setup variables
**Inputs**
- `entity_dir`: This variable represents the entity directory.
- `entity_name`: This variable holds the entity name.
- `spreadsheet_url`: Google Sheets spreadsheet URL.
- `sheet_name_input`: Google Sheets sheet name storing growth data.

**Outputs**
- `output_dir`: Output directory to save file to.
- `file_sales`: Output file name to save as picke.
- `sheet_name_output`: Google Sheets sheet name storing sales qualified leads.

In [9]:
# Inputs
entity_dir = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "entity_dir")
entity_name = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "entity_name")
spreadsheet_url = pload(os.path.join(naas_data_product.OUTPUTS_PATH, "entities", "0"), "abi_spreadsheet")
sheet_name_input = "CONTACTS"

# Outputs
output_dir = os.path.join(entity_dir, "sales-engine", date.today().isoformat())
file_deals = "deals"
sheet_name_output = "DEALS"

## Model

### Get deals

In [3]:
df_init = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name_output)
if not isinstance(df_init, pd.DataFrame):
    df_init = pd.DataFrame()
print("- Deals (init):", len(df_init))
# df_init.head(3)

- Deals (init): 62


### Get contacts view

In [4]:
df_contacts = gsheet.connect(spreadsheet_url).get(sheet_name=sheet_name_input)
if not isinstance(df_contacts, pd.DataFrame):
    df_contacts = pd.DataFrame()
print("- Contacts view:", len(df_contacts))
# df_contacts.head(3)

- Contacts view: 731


### Get entity data

In [5]:
df_entity = gsheet.connect(spreadsheet_url).get(sheet_name="ENTITY").fillna("NA")
df_entity = df_entity[df_entity["ENTITY"] == entity_name].reset_index(drop=True)
print("- Entity:", entity_name)
print("- Entity row:", len(df_entity))
# df_entity.head(1)

- Entity: Jérémy Ravenel
- Entity row: 1


### Create deals database

In [11]:
def get_list(df, column):
    filters = []
    values = df.loc[0, column].split(",")
    if len(values) > 0:
        filters = [v.strip() for v in values]
    return filters

def create_db_deals(
    df_init,
    df_entity,
    entity_name
):
    # Init
    df = df_init.copy()
    
    # Filters on SQL
    df = df[
        (df["ENTITY"].str.contains(entity_name)) & 
        (df["LEAD_STATUS"].isin(["Sales Qualified Lead"]))
    ]
    print("Sales Qualified Lead:", len(df))
    print()
    
    # Apply ICP filters from entity
    for c in df_entity.columns:
        value = df_entity.loc[0, c]
        if c.startswith("ICP"):
            col_icp = c.replace("ICP_", "")
            if value == "NA":
                filter_icp = df[col_icp].unique().tolist()
            else:
                filter_icp = get_list(df_entity, c)
            print(f"- Applying filter on {col_icp}:", filter_icp)
            df = df[(df[col_icp].isin(filter_icp))]
            print("- Remaining deals:", len(df))
            print()
            
    # Create columns
    df["SCENARIO"] = pd.to_datetime(df["SQL_DATE"].str[:19]).dt.strftime("W%W-%Y")
    df["DEAL_NAME"] = df["PEOPLE_FULLNAME"] + " (" + df["ORG_NAME"] + ")"
    seniority_score = {
        "Entry-Level": 1,
        "Professional/Staff": 2,
        "Senior Professional/Staff": 3,
        "Lead/Supervisor": 4,
        "Manager": 5,
        "Senior Manager": 6,
        "Executive": 7,
        "Top Executive": 8
    }
    df["PEOPLE_SENIORITY_SCORE"] = df["PEOPLE_SENIORITY"].map(seniority_score).fillna(0)
    df["DEAL_SCORE"] = df["PEOPLE_INTERACTION_SCORE"].astype(int) + df["ORG_INTERACTION_SCORE"].astype(int) + df["PEOPLE_SENIORITY_SCORE"].astype(int)
    df["SQL_DATE"] = df["SQL_DATE"].str[:10]
    df["CRM_DEAL_ID"] = "TBD"
    
    # Cleaning
    to_order = [
        "ENTITY",
        "SCENARIO",
        "DEAL_NAME",
        "SQL_DATE",
        "DEAL_SCORE",
        "PEOPLE_FULLNAME",
        "PEOPLE_OCCUPATION",
        "PEOPLE_SENIORITY",
        "PEOPLE_DEPARTMENT",
        "PEOPLE_NOTES",
        "MESSAGING_OPTIONS",
        "ORG_NAME",
        "ORG_STAFF_RANGE_NAME",
        "ORG_INDUSTRY",
        'ORG_COUNTRY',
        'PEOPLE_PROFILE_URL',
        'ORG_LINKEDIN_URL',
        "CRM_CONTACT_ID",
        "CRM_ORG_ID",
        "CRM_DEAL_ID"
    ]
    to_rename = {
        "SQL_DATE": "DEAL_CREATED_DATE",
    }
    df = df[to_order].rename(columns=to_rename).sort_values(by=["DEAL_CREATED_DATE", "DEAL_SCORE"], ascending=[False, False])
    return df.reset_index(drop=True)

db_deals = create_db_deals(
    df_contacts,
    df_entity,
    entity_name
)
print("🙌 Deals:", len(db_deals))
db_deals.head(3)

Sales Qualified Lead: 103

- Applying filter on PEOPLE_SENIORITY: ['Professional/Staff', 'Senior Professional/Staff', 'Lead/Supervisor', 'Manager', 'Senior Manager', 'Executive', 'Top Executive']
- Remaining deals: 103

- Applying filter on PEOPLE_DEPARTMENT: ['Human Resources (HR)', 'Finance', 'Marketing', 'Sales', 'Operations', 'Information Technology (IT)', 'Research and Development (R&D)', 'Customer Service', 'Legal', 'Procurement', 'Quality Assurance (QA)', 'Logistics and Supply Chain', 'Public Relations (PR)', 'Executive Management', 'Product Management', 'Strategy and Business Development', 'Education']
- Remaining deals: 80

- Applying filter on ORG_STAFF_RANGE_NAME: ['Solopreneur (0-1)', 'Micro Team (2-10)', 'Small Company (11-50)', 'Medium Company (51-200)', 'Large Company (201-500)', 'Enterprise Level (501-1000)', 'Major Corporation(1001-5000)', 'Global Corporation (5001-10000)', 'Mega Corporation (10001>)']
- Remaining deals: 62

- Applying filter on ORG_NAME: ['Fribl', '33

,ENTITY,SCENARIO,DEAL_NAME,DEAL_CREATED_DATE,DEAL_SCORE,PEOPLE_FULLNAME,PEOPLE_OCCUPATION,PEOPLE_SENIORITY,PEOPLE_DEPARTMENT,PEOPLE_NOTES,MESSAGING_OPTIONS,ORG_NAME,ORG_STAFF_RANGE_NAME,ORG_INDUSTRY,ORG_COUNTRY,PEOPLE_PROFILE_URL,ORG_LINKEDIN_URL,CRM_CONTACT_ID,CRM_ORG_ID,CRM_DEAL_ID
0,Jérémy Ravenel,W04-2024,Anthony Alcaraz (Fribl),2024-01-22,99,Anthony Alcaraz,Chief Product Officer - AI Architect | @Fribl ...,Executive,Product Management,"[""Anthony Alcaraz commented '💕 ' on 'I wish w...","1. Hi Anthony, I noticed your comment on Jérém...",Fribl,Micro Team (2-10),"Technology, Information and Internet",France,https://www.linkedin.com/in/ACoAACVPJNABGwj10w...,https://www.linkedin.com/company/fribl,TBD,TBD,TBD
1,Jérémy Ravenel,W04-2024,Matteo Castiello (33A),2024-01-22,64,Matteo Castiello,Generative AI Advisor and Researcher | Guiding...,Professional/Staff,Research and Development (R&D),"[""Matteo Castiello sent 'praise' reaction to '...","1. Hi Matteo, I noticed your expertise in Gene...",33A,Micro Team (2-10),Design Services,Denmark,https://www.linkedin.com/in/ACoAACenYg8BoVOSWA...,https://www.linkedin.com/company/33a,TBD,TBD,TBD
2,Jérémy Ravenel,W04-2024,Vin Vashishta (V Squared),2024-01-22,57,Vin Vashishta,AI Advisor | Author “From Data To Profit” | Co...,Executive,Education,['Vin Vashishta sent \'like\' reaction to \'We...,"1. ""Hi Vin, I couldn't agree more with your th...",V Squared,Micro Team (2-10),Business Consulting and Services,United States,https://www.linkedin.com/in/ACoAAADS0WQBhQQVMD...,https://www.linkedin.com/company/endgame-engin...,TBD,TBD,TBD


## Output

### Save data

In [12]:
pdump(output_dir, db_deals, file_deals)

### Send data to Google Sheets spreadsheet

In [13]:
df_check = pd.concat([db_deals.astype(str), df_init.astype(str)]).drop_duplicates(keep=False)
if len(df_check) > 0:
    gsheet.connect(spreadsheet_url).send(sheet_name=sheet_name_output, data=db_deals, append=False)
else:
    print("Noting to update in Google Sheets!")   